In [ ]:
# this block will load the housing data as we did before


import os
import tarfile
from six.moves import urllib
import numpy as np

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

fetch_housing_data()
housing = load_housing_data()
housing.head()

In [ ]:
# here are some visualizations of the data that are in this dataset

import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# some more code for visualizing the relationships between two attributes in the dataset

# if you want bigger figures, create another cell below this block with fewer attributes selected

from pandas.plotting import scatter_matrix

attributes = ["latitude", "longitude", "total_bedrooms", "median_house_value", "median_income", "total_rooms",
              "housing_median_age", "population", "households" ]
scatter_matrix(housing[attributes], figsize=(12, 8))


In [ ]:
#now let's try pulling out some items in the dataset (for this, I'll look at proximity to the bay as the output and median income as the input)

X = np.array(housing["longitude"]).reshape(-1,1)
yval = np.array(housing["median_house_value"])
y = ( yval > 200000 ).astype(np.int)

print( X.shape, y.shape )

In [ ]:
# we can look at this as logistic data (1 or 0)

import matplotlib.pyplot as plt
plt.plot( X, y, 'ro')

In [ ]:
# or, we can look at this as linear data

import matplotlib.pyplot as plt
plt.plot( X, yval, 'ro')

In [ ]:
# let's try some logistic regression on the data

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='lbfgs')
log_reg.fit(X,y)

In [ ]:
# you'll want to visualize the output of the logistic regression

# to create the curves, need to sample the space evenly
X_new = np.linspace(-124, -114, 1000).reshape(-1, 1) #make sure to update this to match the range of your xvariable
# predicted probabilities for each of the sampled papers
y_proba = log_reg.predict_proba(X_new)

# plot the data
plt.plot(X,y == 1,'r.')

#plot the curves
plt.plot(X_new, y_proba[:, 1], "g-")
plt.plot(X_new, y_proba[:, 0], "b--")
plt.ylabel('probability')

In [ ]:
# let's also try some linear regression on the data

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

polynomial_features = PolynomialFeatures(degree=1,
                                         include_bias=False)
linear_regression = LinearRegression()
pipeline = Pipeline([("polynomial_features", polynomial_features),
                     ("linear_regression", linear_regression)])
pipeline.fit(X, yval)


In [ ]:
X_test = np.linspace(-124, -114, 1000)
plt.plot(X_test, pipeline.predict(X_test[:, np.newaxis]), label="Model")
plt.scatter(X, yval, edgecolor='b', s=20, label="Samples")
plt.xlabel("x")
plt.ylabel("y")
